In [1]:
from game import Game
import time
import os
import re

## Load maps

In [2]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Tutorial

In [3]:
print("This is an example of the game map:")
map = load_map(2)
game = Game(map)
game.display_map()

This is an example of the game map:
W	P1	H	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [4]:
game.get_box_locations()

[(2, 3), (3, 2), (4, 3)]

In [5]:
game.get_goal_locations()

[(1, 3), (3, 1), (5, 3)]

In [6]:
game.get_player_position()

(0, 2)

- W : Wall
- H : Human
- B : Box
- P : Portal
- G : Goal

In [7]:
for direction in ['U', 'D', 'R', 'L']:
    result = game.apply_move(direction)
    print(f"Move {direction} is valid: {result}")
    if result:
        game.display_map()

Move U is valid: False
Move D is valid: False
Move R is valid: False
Move L is valid: True
W	P1	.	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	H	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [8]:
game.apply_move('U')
game.display_map()

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	H	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [9]:
game.apply_moves(['D', 'L', 'R', 'D']) 
game.display_map()
print("Is game won?", game.is_game_won())

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	.	W	W	W
W	G2/B2	.	.	P1	W	W
W	W	W	H	W	W	W
W	W	W	G3/B3	W	W	W
W	W	W	W	W	W	W
Is game won? True


## Solvers

# helper functions

In [10]:
def state_convertor(state):
    player = state[0]
    boxes = state[1]
    player_list = []
    for co in player:
        player_list.append(co)
    player_tuple = tuple(player_list)
    boxes_list = []
    for box in boxes:
        current_box_list = []
        for co in box:
            current_box_list.append(co)
        current_box_tuple = tuple(current_box_list)
        boxes_list.append(current_box_tuple)
    boxes_tuple = tuple(boxes_list)
    final = (player_tuple,boxes_tuple)
    return final

def wall_or_out(pos, game):
    row = pos[0]
    collomn = pos[1] 
    if row < 0:
        return True
    if row >= game.height:
        return True
    if collomn < 0:
        return True
    if collomn >= game.width:
        return True
    if game.game_map[row][collomn][0] == 'W':
        return True
    return False

def new_possition(pos, move, game):
    deltaX = move[0]
    deltaY = move[1]
    pos_row = pos[0]
    pos_col = pos[1]
    new_row = pos_row + deltaY
    new_col = pos_col + deltaX
    new_pos = [new_row, new_col]
    if new_row >= 0 and new_row < game.height and new_col >= 0 and new_col < game.width:
        if game.game_map[new_row][new_col][0] == 'P':
            portal_id = int(game.game_map[new_row][new_col][1:])          
            portalI = game.portals[portal_id - 1][0]
            portalII = game.portals[portal_id - 1][1]
            portal_pos0 = list(portalI)
            portal_pos1 = list(portalII)
            if new_pos == portal_pos0:
                other = portal_pos1
            else:
                other = portal_pos0
            other_row = other[0]
            other_col = other[1]
            new_other_row = other_row + deltaY
            new_other_col = other_col + deltaX
            new_pos = [new_other_row, new_other_col]
            return new_pos
    return new_pos

def state_expand(state, move, game):
    player = state[0]
    boxes = state[1]
    deltaX = move[0]
    deltaY = move[1]
    new_player = new_possition(player, (deltaY, deltaX), game)
    if wall_or_out(new_player, game):
        return None
    new_boxes = []
    for b in boxes:
        new_box = list(b)
        new_boxes.append(new_box)
    box_moved = False
    for i in range(len(new_boxes)):
        box = new_boxes[i]
        if box == new_player:
            box_moved = True
            new_box = new_possition(box, (deltaY, deltaX), game)
            if wall_or_out(new_box, game):
                return None
            collision_found = False
            for j in range(len(new_boxes)):
                if j != i:
                    other_box = new_boxes[j]
                    if other_box == new_box:
                        collision_found = True
                        break
            if collision_found:
                return None
            new_boxes[i] = new_box
            break
    new_state = [new_player, new_boxes]
    return new_state

def manhattan(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])



### BFS

In [16]:

# TODO: Must return moves (if there is no solution return None), number of visited states

def solver_bfs(game_map):
    
    game = Game(game_map)
    player_initial = list(game.player_pos)
    boxes_initial = []
    for b in game.boxes:
        current_box = list(b)
        boxes_initial.append(current_box)
    initial_state = [player_initial, boxes_initial]
    goal_state = []
    for g in game.goals:
        current_goal = list(g)
        goal_state.append(current_goal)
    visited = {}
    initial_key = state_convertor(initial_state)
    visited[initial_key] = (None, None)
    queue = []
    queue.append(initial_state)
    moves = {}
    moves['L'] = (-1, 0)
    moves['R'] = (1, 0)
    moves['U'] = (0, 1)
    moves['D'] = (0, -1)
    while len(queue) > 0:
        current_state = queue.pop(0)
        current_key = state_convertor(current_state)
        if current_state[1] == goal_state:
            path = []
            key_to_continue = current_key
            while visited[key_to_continue][0] is not None:
                previous_state = visited[key_to_continue][0]
                path.append(visited[key_to_continue][1])
                key_to_continue = state_convertor(previous_state)
            path.reverse()
            return path, len(visited)
        for m in moves:
            movess = moves[m]
            new_state = state_expand(current_state, movess, game)
            if new_state is not None:
                new_key = state_convertor(new_state)
                if new_key not in visited:
                    visited[new_key] = (current_state, m)
                    queue.append(new_state)
    return None, len(visited)

print("BFS algoritm:")
for i in range (1,11) :
    if  i == 9 :
        continue
    start_time = time.time()
    solution, visited_states = solver_bfs(load_map(i))
    end_time = time.time()
    print(f"Moves: {solution} -----  Visited States: {visited_states} ----- map: {i}")
    print(round(end_time - start_time, 4))

BFS algoritm:
Moves: ['L', 'R', 'R', 'L', 'U', 'D', 'D'] -----  Visited States: 47 ----- map: 1
0.0033
Moves: ['D', 'L', 'R', 'R', 'L', 'D'] -----  Visited States: 26 ----- map: 2
0.0011
Moves: ['L', 'D', 'R', 'R', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R'] -----  Visited States: 130 ----- map: 3
0.0043
Moves: None -----  Visited States: 2 ----- map: 4
0.0011
Moves: ['L', 'D', 'R', 'R', 'U', 'R', 'D', 'D', 'D', 'L', 'L', 'L', 'U', 'L', 'D'] -----  Visited States: 7816 ----- map: 5
0.1853
Moves: ['L', 'L', 'L', 'L', 'L', 'U', 'U', 'U', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'U', 'R', 'D', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] -----  Visited States: 20823 ----- map: 6
0.4836
Moves: ['U', 'L', 'U', 'U', 'R', 'R', 'R', 'R', 'D', 'R', 'U', 'L', 'L', 'L', 'L', 'D', 'D', 'D', 'U', 'R', 'U', 'R', 'U', 'R', 'R', 'D', 'D', 'R', 'D', 'D', 'L', 'L', 'R', 'U'] -----  Visited States: 802173 ----- map: 7
40.5897
Moves: ['L', 'L', 'U', 'R', 'D', 'R', 'U', 'U', '

### DFS

In [ ]:
# TODO: Must return moves, number of visited states

def solver_dfs(game_map):
    game = Game(game_map)
    player_initial = list(game.player_pos)
    boxes_initial = []
    for b in game.boxes:
        current_box = list(b)
        boxes_initial.append(current_box)
    initial_state = [player_initial, boxes_initial]
    goal_state = []
    for g in game.goals:
        current_goal = list(g)
        goal_state.append(current_goal)
    visited = {}
    initial_key = state_convertor(initial_state)
    visited[initial_key] = (None, None)
    stack = []
    stack.append(initial_state)
    moves = {}
    moves['L'] = (-1, 0)
    moves['R'] = (1, 0)
    moves['U'] = (0, 1)
    moves['D'] = (0, -1)
    while len(stack) > 0:
        current_state = stack.pop()
        current_key = state_convertor(current_state)
        if current_state[1] == goal_state:
            path = []
            key_to_continue = current_key
            while visited[key_to_continue][0] is not None:
                previous_state = visited[key_to_continue][0]
                path.append(visited[key_to_continue][1])
                key_to_continue = state_convertor(previous_state)
            path.reverse()
            return path, len(visited)
        for m in moves:
            movess = moves[m]
            new_state = state_expand(current_state, movess, game)
            if new_state is not None:
                new_key = state_convertor(new_state)
                if new_key not in visited:
                    visited[new_key] = (current_state, m)
                    stack.append(new_state)
    return None, len(visited)


print("DFS algoritm:")
for i in range (1,9) :
    if i == 7  :
        continue
    start_time = time.time()
    solution, visited_states = solver_dfs(load_map(i))
    end_time = time.time()
    print(f"Moves: {solution} -----  Visited States: {visited_states} ----- map: {i}")
    print(round(end_time - start_time, 4))


### IDS

In [ ]:

# TODO: Must return moves, number of visited states

def dls(state, depth, limit, path, visited, goal_state, game, moves):
    if state[1] == goal_state:
        return path
    if depth == limit:
        return None
    for m in moves:
        move_delta = moves[m]
        new_state = state_expand(state, move_delta, game)
        if new_state is not None:
            new_state_key = state_convertor(new_state)
            if new_state_key not in visited:
                visited.add(new_state_key)
                new_path = list(path)
                new_path.append(m)
                result = dls(new_state, depth + 1, limit, new_path, visited, goal_state, game, moves)
                if result is not None:
                    return result
                visited.remove(new_state_key)
    return None

def solver_ids(game_map):
    game = Game(game_map)
    player_initial = list(game.player_pos)
    boxes_initial = []
    for b in game.boxes:
        current_box = list(b)
        boxes_initial.append(current_box)
    initial_state = [player_initial, boxes_initial]
    goal_state = []
    for g in game.goals:
        current_goal = list(g)
        goal_state.append(current_goal)
    moves = {}
    moves['L'] = (-1, 0)
    moves['R'] = (1, 0)
    moves['U'] = (0, 1)
    moves['D'] = (0, -1)
    limit = 0
    while 1:
        visited = set()
        initial_key = state_convertor(initial_state)
        visited.add(initial_key)
        result = dls(initial_state, 0, limit, [], visited, goal_state, game, moves)
        if result is not None:
            return result, len(result)
        limit = limit + 1
        
        
print("IDS algoritm:")
for i in range (1,9) :
    if i == 4 or i == 7 or i == 6 :
        continue
    start_time = time.time()
    solution, visited_states = solver_ids(load_map(i))
    end_time = time.time()
    print(f"Moves: {solution} -----  Visited States: {visited_states} ----- map: {i}")
    print(round(end_time - start_time, 4))


### A*

In [13]:
#TODO
def heuristic(state, game):
    boxes = state[1]
    total_heu = 0
    for i in range(len(boxes)):
        box = boxes[i]
        goal = game.goals[i]
        d_direct = manhattan(box, goal)
        d_one = float('inf')
        for portal in game.portals:
            p1, p2 = portal
            option1 = manhattan(box, p1) + manhattan(p2, goal)
            option2 = manhattan(box, p2) + manhattan(p1, goal)
            d_one = min(d_one, option1, option2)
        d_two = float('inf')
        if len(game.portals) >= 2:
            for j in range(len(game.portals)):
                for k in range(len(game.portals)):
                    if j == k:
                        continue
                    p1, p2 = game.portals[j]
                    q1, q2 = game.portals[k]
                    option1 = manhattan(box, p1) + manhattan(p2, q1) + manhattan(q2, goal)
                    option2 = manhattan(box, p1) + manhattan(p2, q2) + manhattan(q1, goal)
                    option3 = manhattan(box, p2) + manhattan(p1, q1) + manhattan(q2, goal)
                    option4 = manhattan(box, p2) + manhattan(p1, q2) + manhattan(q1, goal)
                    d_two = min(d_two, option1, option2, option3, option4)
        
        best_cost = min(d_direct, d_one, d_two)
        total_heu += best_cost
    return total_heu


# # TODO: Must return moves, number of visited states
def solver_astar(game_map, heuristic_func=heuristic, weight=1):
    game = Game(game_map)
    
    player_initial = list(game.player_pos)
    boxes_initial = []
    for box in game.boxes:
        boxes_initial.append(list(box))
    initial_state = [player_initial, boxes_initial]
    goal_state = []
    for goal in game.goals:
        goal_state.append(list(goal))
    initial_key = state_convertor(initial_state)
    visited = {initial_key: (0, None, None)}
    h_initial = heuristic_func(initial_state, game)
    last = [(h_initial, 0, initial_state)]
    moves = {}
    moves['L'] = (-1, 0)
    moves['R'] = (1, 0)
    moves['U'] = (0, 1)
    moves['D'] = (0, -1)
    while len(last) > 0:
        best_index = 0
        best_f, best_g, best_state = last[0]
        for index in range(1, len(last)):
            current_f, current_g, current_state = last[index]
            if current_f < best_f:
                best_f = current_f
                best_g = current_g
                best_state = current_state
                best_index = index
        last.pop(best_index)
        current_key = state_convertor(best_state)
        if best_state[1] == goal_state:
            path = []
            key = current_key
            while visited[key][1] is not None:
                parent_state = visited[key][1]
                move_taken = visited[key][2]
                path.append(move_taken)
                key = state_convertor(parent_state)
            path.reverse()
            return path, len(visited)
        for m in moves:
            delta = moves[m]
            new_state = state_expand(best_state, delta, game)
            if new_state is not None:
                new_key = state_convertor(new_state)
                new_g = best_g + 1  
                if new_key not in visited or new_g < visited[new_key][0]:
                    visited[new_key] = (new_g, best_state, m)
                    h_value = heuristic_func(new_state, game)
                    new_f = new_g + weight * h_value
                    last.append((new_f, new_g, new_state))
    
    return None, len(visited)


print("A* algoritm:")
for i in range (1,11) :
    if i == 9 or i == 7:
        continue
    start_time = time.time()
    solution, visited_states = solver_astar(load_map(i))
    end_time = time.time()
    print(f"Weight: {1} ----- Moves: {solution} -----  Visited States: {visited_states} ----- map: {i}")
    print(round(end_time - start_time, 4))


# print("weight A* algoritm:")
# for i in range (1,11) :
#     start_time = time.time()
#     solution, visited_states = solver_astar(load_map(i),weight=4.2)
#     end_time = time.time()
#     print(f"Weight: {4.2} ----- Moves: {solution} -----  Visited States: {visited_states} ----- map: {i}")
#     print(round(end_time - start_time, 4))


A* algoritm:
Weight: 1 ----- Moves: ['L', 'R', 'R', 'L', 'U', 'D', 'D'] -----  Visited States: 47 ----- map: 1
0.0047
Weight: 1 ----- Moves: ['D', 'L', 'R', 'R', 'L', 'D'] -----  Visited States: 26 ----- map: 2
0.0019
Weight: 1 ----- Moves: ['L', 'D', 'R', 'R', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R'] -----  Visited States: 114 ----- map: 3
0.0
Weight: 1 ----- Moves: None -----  Visited States: 2 ----- map: 4
0.0
Weight: 1 ----- Moves: ['D', 'L', 'D', 'R', 'R', 'U', 'R', 'D', 'D', 'L', 'L', 'L', 'U', 'L', 'D'] -----  Visited States: 1269 ----- map: 5
0.0593
Weight: 1 ----- Moves: ['L', 'L', 'L', 'L', 'L', 'U', 'U', 'U', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'U', 'R', 'D', 'U', 'U', 'U', 'U', 'U', 'U', 'U'] -----  Visited States: 8246 ----- map: 6
1.8236
Weight: 1 ----- Moves: ['L', 'L', 'U', 'R', 'D', 'R', 'U', 'U', 'R', 'U', 'L', 'U', 'R', 'U'] -----  Visited States: 823 ----- map: 8
0.0507
Weight: 1 ----- Moves: ['U', 'U', 'U', 'U', 'U', 

## Solve

In [60]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar,
}

In [61]:
def solve(map, method):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    moves, numof_visited_states = SOLVERS[method](game_map)
    end_time = time.time()
    print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")
            

In [62]:
# solve(1, "IDS") # Solve map 1 using IDS

In [ ]:
# def solve_all():
#     for map in range(min(map_file_indices), max(map_file_indices) + 1):
#         for method in SOLVERS.keys():
#             solve(map, method)
            

In [64]:
# solve_all() # Solve all maps using all methods